In [ ]:
def group_plot(df, x, y, group, error_x=None, error_y=None, fig=None, ax_=None):
    cmap = plt.cm.tab10
    colour_palette = list()
    for i in range(cmap.N):
        colour_palette.append(cmap(i))
    
    if fig is None and ax_ is None:
        fig, ax = plt.subplots()
    else:
        ax = ax_
    for i, (group_label, group_df) in enumerate(df.groupby(group)):
        if error_x is not None or error_y is not None:
            ax.errorbar(
                group_df.loc[:, x], 
                group_df.loc[:, y],
                xerr=group_df[error_x],
                yerr=group_df[error_y],
                ecolor=colour_palette[i],
                fmt="none",
                marker=None,
                zorder=1,
                lw=1 # Required to get error bars centered. Classic matplotlib
                )
        ax.scatter(
            group_df.loc[:, x], 
            group_df.loc[:, y],
            color=colour_palette[i], 
            label=group_label,
            zorder=2
        )

    ax.set_title(group, size=15)
    ax.legend()
    # ax.set_xscale("log")
    ax.set_xlabel(x, fontsize=15)
    ax.set_ylabel(y, fontsize=15)
    plt.tight_layout()
    fig.set_facecolor("white")
    plt.subplots_adjust( 
                    wspace=0.2,
                    hspace=0.2)

In [ ]:
group_columns = ["Vendor", "Modality", "sites", "Sites-SubSampled", 
                "Binning", "Magnification", "Number_of_channels", "z_plane", 
                "BF_Zplanes"]
subplots = np.arange(len(group_columns)).reshape(-1,3).shape

fig, ax = plt.subplots(subplots[0], subplots[1], figsize=(17,17))
for i, _ax in zip(group_columns, ax.flatten()):
    group_plot(df_replicating[df_replicating["sphering"] == True], "cell_count", "Percent_Replicating", group=i, fig=fig, ax_=_ax)
fig.savefig("figures/rep_vs_cell-count_sphering_TRUE.png")


In [ ]:
group_columns = ["Vendor", "Modality", "Images_per_well", "Sites-SubSampled", 
                "Binning", "Magnification", "Number_of_channels", "z_plane", 
                "BF_Zplanes"]
subplots = np.arange(len(group_columns)).reshape(-1,3).shape

fig, ax = plt.subplots(subplots[0], subplots[1], figsize=(17,17))
for i, _ax in zip(group_columns, ax.flatten()):
    group_plot(df_replicating[df_replicating["sphering"] == False], "cell_count", "Percent_Replicating", group=i, fig=fig, ax_=_ax)
fig.savefig("figures/rep_vs_cell-count_sphering_FALSE.png")


In [ ]:
group_columns = ["Vendor", "Modality", "spinning-disc", "Binning", 
                "Magnification", "aperture", "dry-immersion",
                "Number_of_channels", "Images_per_well",
                "z_plane", "BF_Zplanes"]

fig, ax = plt.subplots(4,3, figsize=(20,20))
for i, _ax in zip(group_columns, ax.flatten()):
    group_plot(match_rep_df[match_rep_df["sphering"] == False], "Percent_Matching", "Percent_Replicating", group=i, fig=fig, ax_=_ax)
fig.savefig("figures/rep_vs_match_groupings_sphering_FALSE.png")


In [ ]:
group_columns = ["Vendor", "Modality", "spinning-disc", "Binning", 
                "Magnification", "aperture", "dry-immersion",
                "Number_of_channels", "Images_per_well",
                "z_plane", "BF_Zplanes"]

fig, ax = plt.subplots(4,3, figsize=(20,20))
for i, _ax in zip(group_columns, ax.flatten()):
    group_plot(match_rep_df[match_rep_df["sphering"] == True], "Percent_Matching", "Percent_Replicating", group=i, fig=fig, ax_=_ax)
fig.savefig("figures/rep_vs_match_groupings_sphering_TRUE.png")

# Creating mean and SD columns for grouped DF

In [ ]:
def find_group_avg_df(_df, group):
    df = _df.copy()
    df['Mean_Percent_Replicating'] = list(df['Percent_Replicating'])
    df['SD_Percent_Replicating'] = list(df['Percent_Replicating'])
    df['Mean_Percent_Matching'] = list(df['Percent_Matching'])
    df['SD_Percent_Matching'] = list(df['Percent_Matching'])
    group_df = df.groupby(
                            group,as_index=False).agg({
                                'Percent_Replicating' : lambda x: list(x),
                                'Mean_Percent_Replicating' : lambda y: np.mean(y),
                                'SD_Percent_Replicating' : lambda z: float('%.3f'%np.std(z)),
                                'Percent_Matching' : lambda x: list(x),
                                'Mean_Percent_Matching' : lambda y: np.mean(y),
                                'SD_Percent_Matching' : lambda z: float('%.3f'%np.std(z))
                                })
    return group_df

In [ ]:
group_columns = ["Vendor", ["Modality", "z_plane"], "Images_per_well", 
                "Binning", "Magnification", "Number_of_channels", "z_plane", 
                "BF_Zplanes", "aperture", "dry-immersion", "spinning-disc", 
                "simultaneous-excitation", "vs-brightfield", "sites", None]
# group_columns = [["Modality", "z_plane"], "Vendor", "Number_of_channels"]

subplots = np.arange(len(group_columns)).reshape(-1,3).shape
fig, ax = plt.subplots(subplots[0], subplots[1], figsize=(20,30))
for i, _ax in zip(group_columns, ax.flatten()):
    try:
        group_df = find_group_avg_df(match_rep_df.loc[match_rep_df["sphering"]==True], i)
        group_plot(
            df=group_df, 
            x="Mean_Percent_Matching", 
            y="Mean_Percent_Replicating",
            group=i,
            error_x="SD_Percent_Replicating",
            error_y="SD_Percent_Matching",
            fig=fig, 
            ax_=_ax)
    except:
        fig.delaxes(_ax)
fig.savefig("figures/mean-for-ALL-scope-groupings_sphering_TRUE.png")

In [ ]:
group_columns = ["Vendor", ["Modality", "z_plane"], "Images_per_well", 
                "Binning", "Magnification", "Number_of_channels", "z_plane", 
                "BF_Zplanes", "aperture", "dry-immersion", "spinning-disc", 
                "simultaneous-excitation", "vs-brightfield", "sites", None]
# group_columns = [["Modality", "z_plane"], "Vendor", "Number_of_channels"]

subplots = np.arange(len(group_columns)).reshape(-1,3).shape
fig, ax = plt.subplots(subplots[0], subplots[1], figsize=(20,30))
for i, _ax in zip(group_columns, ax.flatten()):
    try:
        group_df = find_group_avg_df(match_rep_df.loc[match_rep_df["sphering"]==False], i)
        group_plot(
            df=group_df, 
            x="Mean_Percent_Matching", 
            y="Mean_Percent_Replicating",
            group=i,
            error_x="SD_Percent_Replicating",
            error_y="SD_Percent_Matching",
            fig=fig, 
            ax_=_ax)
    except:
        fig.delaxes(_ax)
fig.savefig("figures/mean-for-ALL-scope-groupings_sphering_FALSE.png")